In [1]:
pip install pandas openpyxl


In [60]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the life insurance data
file_path = 'Health_Insurance.xlsx'
df = pd.read_excel(file_path)

# Calculate Z-scores for 'Income' and 'Assets' columns
df['Z_Score_Income'] = (df['Income'] - df['Income'].mean()) / df['Income'].std()
df['Z_Score_Assets'] = (df['Assets'] - df['Assets'].mean()) / df['Assets'].std()

# Define the conditions for mismatched applicants
mismatch_condition = (
    (((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2)) & (df['Age'] < 21)) |  # Z-score for Income or Assets greater than 2 and Age less than 21
    ((df['Profession'].isin(['driver', 'technician', 'operator'])) & ((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2))) |  # Profession is 'driver', 'technician', 'operator' and Z-score for Income greater than 2 or Z-score for Assets greater than 2
    ((df['Age'] < 21) & (df['State'] == 'MH'))  # Age is less than 21 and Location is Rural
)

# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = False
df.loc[mismatch_condition, 'Mismatched_Applicant'] = True

# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants_Output.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')

    # Highlight mismatched applicants in the output sheet
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    # Get the column index for 'Mismatched_Applicant'
    mismatch_column_index = df.columns.get_loc('Mismatched_Applicant') + 1

    for row in output_sheet.iter_rows(min_row=2, max_row=output_sheet.max_row, min_col=mismatch_column_index, max_col=mismatch_column_index):
        for cell in row:
            if cell.value:
                cell.fill = fill

print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants_Output.xlsx


In [62]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read the life insurance data
file_path = 'Health_Insurance.xlsx'
df = pd.read_excel(file_path)

# Define the conditions for mismatched applicants
mismatch_condition = (
    (((df['Income'] > 4000000) | (df['Assets'] > 10000000)) & (df['Age'] <= 21)) |  # Income or assets greater than 4 crore and age less than or equal to 21
    ((df['Profession'].isin(['driver', 'technician', 'operator'])) & (df['Income'] > 2000000) | (df['Assets'] > 10000000)) |  # Profession is 'driver', 'technician', 'operator' and income greater than 4 crore
    ((df['Age'] < 21) & (df['State'] == 'MH'))  # Location MH
)

# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = False
df.loc[mismatch_condition, 'Mismatched_Applicant'] = True

# Save the results to a new Excel file with highlighted mismatched applicants
output_excel_file_path = 'Mismatched_Applicants.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')

    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    # Get the column index for 'Mismatched_Applicant'
    mismatch_column_index = df.columns.get_loc('Mismatched_Applicant') + 1

    for row in output_sheet.iter_rows(min_row=2, max_row=output_sheet.max_row, min_col=mismatch_column_index, max_col=mismatch_column_index):
        for cell in row:
            if cell.value:
                cell.fill = fill

print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")


Mismatched applicants flagged and saved to Mismatched_Applicants.xlsx


# Isolatio Forest

In [63]:
# Isolatio Forest
import pandas as pd 
import numpy as np

In [64]:
df = pd.read_excel('Health_Insurance.xlsx')
df

,Application ID,Name,Gender,Age,Profession,Income,Assets,Location,State,City,Health Condition
0,1,Arjun,Male,30,technician,6000000,12000000,Pune,MH,Pune,Good
1,2,Kavya,Female,45,Engineer,75000,15000000,Mumbai,MH,Mumbai,Excellent
2,3,Rajat,Male,19,Teacher,500000,20000000,Sambhajinagar,MH,Coimbatore,Fair
3,4,Sneha,Female,35,analyst,80000,180000,Pune,MH,Pune,Good
4,5,Akash,Male,50,technician,35000,120000,Pune,MH,Ludhiana,Excellent
5,6,Anaya,Female,32,driver,5000000,60000000,Mumbai,MH,Madurai,Poor
6,7,Vignesh,Male,40,Teacher,550000,11000000,Nashik,MH,Mangalore,Good
7,8,Niharika,Female,28,Lawyer,700000,160000,Daultabad,MH,Nagpur,Excellent
8,9,Pradeep,Male,16,Doctor,1000000,250000,Kannad,MH,Vijayawada,Good
9,10,Anand,Male,38,Engineer,85000,190000,Karodi,MH,Mysuru,Excellent


In [68]:
pip install xgboost


                                              0.0/99.8 MB ? eta -:--:--
                                             0.0/99.8 MB 653.6 kB/s eta 0:02:33
                                              0.2/99.8 MB 2.0 MB/s eta 0:00:51
                                              0.4/99.8 MB 2.5 MB/s eta 0:00:40
                                              0.7/99.8 MB 3.8 MB/s eta 0:00:27
                                              1.4/99.8 MB 6.2 MB/s eta 0:00:16
                                              2.2/99.8 MB 7.9 MB/s eta 0:00:13
     -                                        2.9/99.8 MB 8.9 MB/s eta 0:00:11
     -                                        3.6/99.8 MB 10.0 MB/s eta 0:00:10
     -                                        4.3/99.8 MB 10.1 MB/s eta 0:00:10
     --                                       5.0/99.8 MB 10.7 MB/s eta 0:00:09
     --                                       5.8/99.8 MB 11.2 MB/s eta 0:00:09
     --                                       6.3/99.8

In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from openpyxl.styles import PatternFill  # Import PatternFill class from openpyxl.styles
 
# Load the data
df = pd.read_excel('Health_Insurance.xlsx')  # Replace 'your_data.xlsx' with your actual file path
 
# Initialize global variables
var1 = "No Error"
 
# Feature engineering
df['Location'] = df['State']  # Assuming 'Location' is derived from 'State' (adjust if needed)
# Calculate Z-scores for 'Income' and 'Assets' columns
df['Z_Score_Income'] = (df['Income'] - df['Income'].mean()) / df['Income'].std()
df['Z_Score_Assets'] = (df['Assets'] - df['Assets'].mean()) / df['Assets'].std()
 
# Define conditions for mismatched applicants
mismatch_condition = (
    (((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2)) & (df['Age'] < 21)) |  # Z-score for Income or Assets greater than 2 and Age less than 21
    ((df['Profession'].isin(['driver', 'technician', 'operator'])) & ((df['Z_Score_Income'] > 2) | (df['Z_Score_Assets'] > 2))) |  # Profession is 'driver', 'technician', 'operator' and Z-score for Income greater than 2 or Z-score for Assets greater than 2
    ((df['Age'] < 21) & (df['State'] == 'MH'))  # Age is less than 21 and Location is Rural
)
 
# Create a new column 'Mismatched_Applicant' to flag mismatched applicants
df['Mismatched_Applicant'] = False
df.loc[mismatch_condition, 'Mismatched_Applicant'] = True
 
# Feature scaling
scaler = StandardScaler()
df[['Income', 'Assets', 'Age']] = scaler.fit_transform(df[['Income', 'Assets', 'Age']])
 
# Isolation Forest for outlier detection
isolation_forest = IsolationForest(contamination=0.05)  # Adjust contamination based on your dataset
df['isolation_forest_score'] = isolation_forest.fit_predict(df[['Income', 'Assets', 'Age']])
 
# KNN for further refinement
knn = NearestNeighbors(n_neighbors=10)  # You can adjust the number of neighbors
knn.fit(df[['Income', 'Assets', 'Age']])
distances, indices = knn.kneighbors(df[['Income', 'Assets', 'Age']])
mean_distance = distances.mean(axis=1)
df['knn_distance'] = mean_distance
 
# Flag doubtful applicants based on both Isolation Forest and KNN
df['is_outlier'] = (df['isolation_forest_score'] == -1) & (df['knn_distance'] > df['knn_distance'].quantile(0.95))
 
# Highlight mismatched applicants in Excel
output_excel_file_path = 'Mismatched_Applicants0000.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Mismatched_Applicants')
 
    output_sheet = writer.sheets['Mismatched_Applicants']
    fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
 
    # Get the column index for 'Mismatched_Applicant'
    mismatch_column_index = df.columns.get_loc('Mismatched_Applicant') + 1
 
    for row in output_sheet.iter_rows(min_row=2, max_row=output_sheet.max_row, min_col=mismatch_column_index, max_col=mismatch_column_index):
        for cell in row:
            if cell.value or (cell.column_letter == 'C' and df.at[cell.row-1, 'Age'] < 21):  # Assuming 'Age' column is in column 'C' (adjust if needed)
                cell.fill = fill
 
print(f"Mismatched applicants flagged and saved to {output_excel_file_path}")

Mismatched applicants flagged and saved to Mismatched_Applicants0000.xlsx


In [3]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import NearestNeighbors
 
# Load the data
data = pd.read_excel('Health_Insurance.xlsx')  # Replace 'Life_Insurance_data.xlsx' with your actual file path
 
# Create DataFrame
df = pd.DataFrame(data)
 
# Select relevant features for analysis
features = ['Income', 'Assets']
 
# Initialize Isolation Forest model
isolation_forest = IsolationForest(contamination=0.05)  # Contamination parameter can be adjusted
 
# Fit model and predict outliers
df['IsOutlier'] = isolation_forest.fit_predict(df[features])
 
# Flag applications with outliers
df['Flagged'] = df['IsOutlier'].apply(lambda x: 'Yes' if x == -1 else 'No')
 
# Initialize KNN model for refinement
refinement_features = ['Income', 'Assets', 'Age']
knn = NearestNeighbors(n_neighbors=5)  # Adjust number of neighbors as needed
 
# Fit KNN model
knn.fit(df[refinement_features])
 
# Compute average distance to neighbors
distances, _ = knn.kneighbors(df[refinement_features])
df['MeanDistance'] = distances.mean(axis=1)
 
# Flag applications with unusually high distance to neighbors
threshold = df['MeanDistance'].quantile(0.95)
df['RefinedFlag'] = df['MeanDistance'].apply(lambda x: 'Yes' if x > threshold else 'No')
 
# Save DataFrame to Excel with refined flag
output_file = 'Life_Insurance_data_with_flags.xlsx'
df.to_excel(output_file, index=False)
 
print(f"Data with flags saved to {output_file}")

Data with flags saved to Life_Insurance_data_with_flags.xlsx


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.neighbors import NearestNeighbors

# Load the data
data = pd.read_excel('Health_Insurance.xlsx')  # Replace with your actual file path
df = pd.DataFrame(data)

# Select relevant features for analysis
features = ['Income', 'Assets']

# Initialize Isolation Forest model
isolation_forest = IsolationForest(contamination=0.05)  # Contamination parameter can be adjusted

# Fit model and predict outliers
df['IsOutlier'] = isolation_forest.fit_predict(df[features])

# Map the labels from (-1, 1) to (0, 1)
df['IsOutlier'] = np.where(df['IsOutlier'] == -1, 1, 0)

# Flag applications with outliers
df['Flagged'] = df['IsOutlier'].apply(lambda x: 'Yes' if x == 1 else 'No')

# Encode remaining categorical columns using pd.get_dummies
categorical_columns = ['Gender', 'State', 'City', 'Health Condition', 'Name', 'Profession', 'Location']
df = pd.get_dummies(df, columns=categorical_columns)

# Split the data for XGBoost model
X = df.drop(['IsOutlier', 'Flagged'], axis=1)
y = df['IsOutlier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# Set parameters for XGBoost model
params = {'objective': 'binary:logistic', 'eval_metric': 'logloss', 'use_label_encoder': False}

# Train XGBoost Model
model = xgb.train(params, dtrain, num_boost_round=100)

# Predictions on the test set
y_pred_xgb = model.predict(dtest)

# Convert probabilities to binary predictions
y_pred_xgb_binary = np.where(y_pred_xgb > 0.5, 1, 0)

# Calculate accuracy for XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb_binary)
print(f"XGBoost Accuracy: {accuracy_xgb}")

# Initialize KNN model for refinement
refinement_features = ['Income', 'Assets', 'Age']
knn_refinement = NearestNeighbors(n_neighbors=5)
knn_refinement.fit(X)

# Compute average distance to neighbors
distances_refinement, _ = knn_refinement.kneighbors(X)
df['MeanDistanceRefinement'] = distances_refinement.mean(axis=1)

# Flag applications with unusually high distance to neighbors in the refined dataset
threshold_refinement = df['MeanDistanceRefinement'].quantile(0.95)
df['RefinedFlag'] = df['MeanDistanceRefinement'].apply(lambda x: 'Yes' if x > threshold_refinement else 'No')

# Save DataFrame to Excel with refined flag
output_file = 'Health_Insurance_with_flags.xlsx'
df.to_excel(output_file, index=False)

print(f"Data with flags saved to {output_file}")


C:\ProgramData\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:39:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 1.0
Data with flags saved to Health_Insurance_with_flags.xlsx
